# 🇻🇳 Vietnamese Paraphrase Identification — Inference Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vmhdaica/vietnamese-paraphrase-identification/blob/main/notebooks/inference_demo.ipynb)

This notebook loads the fine-tuned **PhoBERT-base-v2** model from HuggingFace Hub and lets you:
1. Run **batch predictions** on sample sentence pairs
2. Launch an **interactive Gradio UI** right inside Colab

**Model:** [`vmhdaica/vnpi_model_checkpoint_3135`](https://huggingface.co/vmhdaica/vnpi_model_checkpoint_3135)  
**Accuracy:** 97.02% | **Macro-F1:** 0.876 | **PR-AUC:** 0.9995

---

## 1. Install Dependencies

In [ ]:
!pip -q install transformers torch gradio

## 2. Load Model from HuggingFace Hub

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

MODEL_ID = "vmhdaica/vnpi_model_checkpoint_3135"
MAX_LENGTH = 256

print(f"Loading model: {MODEL_ID}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device).eval()
print(f"✅ Model loaded on {device}")
print(f"   Labels: {model.config.id2label}")

## 3. Prediction Function

In [ ]:
def predict_pair(s1: str, s2: str) -> dict:
    """Predict whether two Vietnamese sentences are paraphrases."""
    inputs = tokenizer(
        s1, s2,
        truncation=True, max_length=MAX_LENGTH,
        return_tensors="pt",
    ).to(device)
    inputs.pop("token_type_ids", None)  # PhoBERT/RoBERTa doesn't use this

    with torch.no_grad():
        logits = model(**inputs).logits
        probs = torch.softmax(logits, dim=-1)[0].cpu().numpy()

    label = "✅ Paraphrase" if probs[1] > probs[0] else "❌ Not paraphrase"
    return {
        "label": label,
        "p_paraphrase": round(float(probs[1]), 4),
        "p_not_paraphrase": round(float(probs[0]), 4),
    }

# Quick test
result = predict_pair("Hôm nay trời mưa rất to.", "Thời tiết hôm nay mưa lớn.")
print(result)

## 4. Batch Predictions — Sample Pairs

In [ ]:
import pandas as pd

test_pairs = [
    ("Hôm nay trời mưa rất to.",
     "Thời tiết hôm nay mưa lớn."),
    ("Giá vàng tăng mạnh.",
     "Trận đấu tối qua rất hấp dẫn."),
    ("Thủ tướng đã họp với các bộ trưởng.",
     "Cuộc họp của Thủ tướng với nội các đã diễn ra."),
    ("Hà Nội là thủ đô của Việt Nam.",
     "TP.HCM là thành phố lớn nhất Việt Nam."),
    ("Cô ấy rất giỏi tiếng Anh.",
     "Khả năng tiếng Anh của cô ấy rất tốt."),
    ("Tôi đi ăn phở sáng nay.",
     "Sáng nay tôi đã thưởng thức một tô phở."),
    ("Việt Nam nằm ở Đông Nam Á.",
     "Đất nước Việt Nam thuộc khu vực Đông Nam Á."),
    ("Con mèo ngồi trên bàn.",
     "Chiếc xe đang chạy trên đường."),
]

results = []
for s1, s2 in test_pairs:
    r = predict_pair(s1, s2)
    results.append({
        "Sentence 1": s1,
        "Sentence 2": s2,
        "Prediction": r["label"],
        "P(paraphrase)": r["p_paraphrase"],
    })

df = pd.DataFrame(results)
df.style.set_properties(**{"text-align": "left"})

## 5. Try Your Own Sentences

In [ ]:
# ✏️ Change these sentences and re-run this cell!

sentence_1 = "Việt Nam có nhiều cảnh đẹp."
sentence_2 = "Đất nước Việt Nam rất nhiều phong cảnh tuyệt vời."

result = predict_pair(sentence_1, sentence_2)
print(f"  Sentence 1: {sentence_1}")
print(f"  Sentence 2: {sentence_2}")
print(f"  → {result['label']}  (confidence: {result['p_paraphrase']:.4f})")

## 6. 🎨 Interactive Gradio Demo

Run this cell to launch a **live UI** directly inside Colab!

In [ ]:
import gradio as gr

def gradio_predict(s1: str, s2: str) -> dict:
    if not s1.strip() or not s2.strip():
        return {"paraphrase": 0.0, "not_paraphrase": 1.0}
    inputs = tokenizer(s1, s2, truncation=True, max_length=MAX_LENGTH, return_tensors="pt").to(device)
    inputs.pop("token_type_ids", None)
    with torch.no_grad():
        probs = torch.softmax(model(**inputs).logits, dim=-1)[0].cpu().numpy()
    return {"paraphrase": float(probs[1]), "not_paraphrase": float(probs[0])}

examples = [
    ["Hôm nay trời mưa rất to.", "Thời tiết hôm nay mưa lớn."],
    ["Giá vàng tăng mạnh.", "Trận đấu tối qua rất hấp dẫn."],
    ["Thủ tướng đã họp với các bộ trưởng.", "Cuộc họp của Thủ tướng với nội các đã diễn ra."],
    ["Hà Nội là thủ đô của Việt Nam.", "TP.HCM là thành phố lớn nhất Việt Nam."],
    ["Cô ấy rất giỏi tiếng Anh.", "Khả năng tiếng Anh của cô ấy rất tốt."],
    ["Tôi đi ăn phở sáng nay.", "Sáng nay tôi đã thưởng thức một tô phở."],
]

demo = gr.Interface(
    fn=gradio_predict,
    inputs=[
        gr.Textbox(label="Câu 1 / Sentence 1", placeholder="Nhập câu tiếng Việt...", lines=2),
        gr.Textbox(label="Câu 2 / Sentence 2", placeholder="Nhập câu tiếng Việt...", lines=2),
    ],
    outputs=gr.Label(label="Result", num_top_classes=2),
    title="🇻🇳 Vietnamese Paraphrase Identification",
    description="Compare two Vietnamese sentences. Model: PhoBERT-base-v2 fine-tuned on 40K+ pairs — 97.02% accuracy.",
    examples=examples,
    theme=gr.themes.Soft(),
)

demo.launch(share=True)  # share=True creates a public URL

---

**Resources:**
- 🤗 Model: [`vmhdaica/vnpi_model_checkpoint_3135`](https://huggingface.co/vmhdaica/vnpi_model_checkpoint_3135)
- 📂 GitHub: [`vietnamese-paraphrase-identification`](https://github.com/Hoang-ca/vietnamese-paraphrase-identification)
- 📝 Model Card: see `MODEL_CARD.md` in the repo